In [2]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

model = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

c:\Langgraph\Agentic_by_langgraph\Sequential_workflow\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str

In [4]:
def generate_joke(state:JokeState):
    prompt=f"generate a joke on topic {state['topic']}"
    response=model.invoke(prompt).content
    return {'joke':response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph=StateGraph(JokeState)
graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!',
 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for Pizza):**\n    *   In the context of pizza, "dough" refers to the mixture of flour, water, yeast, and other ingredients that is kneaded and flattened to form the base or crust of the pizza. A pizza *needs* dough to exist.\n\n2.  **"Dough" (Meaning 2 - Slang for Money):**\n    *   In colloquial English, "dough" is a common slang term for "money" or "cash." When people "make some dough," they are earning money.\n\n**The Joke\'s Humor:**\n\nThe joke plays on the double meaning of "dough."\n\n*   When you ask "Why did the pizza get a job?", the expected human answer for getting a job is to earn money.\n*   The punchlin

In [8]:
workflow.get_state(config1)


StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!', 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for Pizza):**\n    *   In the context of pizza, "dough" refers to the mixture of flour, water, yeast, and other ingredients that is kneaded and flattened to form the base or crust of the pizza. A pizza *needs* dough to exist.\n\n2.  **"Dough" (Meaning 2 - Slang for Money):**\n    *   In colloquial English, "dough" is a common slang term for "money" or "cash." When people "make some dough," they are earning money.\n\n**The Joke\'s Humor:**\n\nThe joke plays on the double meaning of "dough."\n\n*   When you ask "Why did the pizza get a job?", the expected human answer for getting a job is to earn money

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!', 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for Pizza):**\n    *   In the context of pizza, "dough" refers to the mixture of flour, water, yeast, and other ingredients that is kneaded and flattened to form the base or crust of the pizza. A pizza *needs* dough to exist.\n\n2.  **"Dough" (Meaning 2 - Slang for Money):**\n    *   In colloquial English, "dough" is a common slang term for "money" or "cash." When people "make some dough," they are earning money.\n\n**The Joke\'s Humor:**\n\nThe joke plays on the double meaning of "dough."\n\n*   When you ask "Why did the pizza get a job?", the expected human answer for getting a job is to earn mone

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)
workflow.get_state(config1)

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!', 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for Pizza):**\n    *   In the context of pizza, "dough" refers to the mixture of flour, water, yeast, and other ingredients that is kneaded and flattened to form the base or crust of the pizza. A pizza *needs* dough to exist.\n\n2.  **"Dough" (Meaning 2 - Slang for Money):**\n    *   In colloquial English, "dough" is a common slang term for "money" or "cash." When people "make some dough," they are earning money.\n\n**The Joke\'s Humor:**\n\nThe joke plays on the double meaning of "dough."\n\n*   When you ask "Why did the pizza get a job?", the expected human answer for getting a job is to earn mone

# Time Travel

In [11]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!', 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for Pizza):**\n    *   In the context of pizza, "dough" refers to the mixture of flour, water, yeast, and other ingredients that is kneaded and flattened to form the base or crust of the pizza. A pizza *needs* dough to exist.\n\n2.  **"Dough" (Meaning 2 - Slang for Money):**\n    *   In colloquial English, "dough" is a common slang term for "money" or "cash." When people "make some dough," they are earning money.\n\n**The Joke\'s Humor:**\n\nThe joke plays on the double meaning of "dough."\n\n*   When you ask "Why did the pizza get a job?", the expected human answer for getting a job is to earn mone

# updating state

In [15]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0f5425-e097-6ffb-8000-bcc00ac958b9'}}

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f5425-e097-6ffb-8000-bcc00ac958b9'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-19T14:23:04.281283+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='d5e03a9d-67b0-f355-6019-6d07232de75a', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it needed to make some **dough**!', 'explanation': 'This is a classic example of a **pun**, which is a joke exploiting the different possible meanings of a word or the fact that there are words that sound alike but have different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meanin

# fault tolerance

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


c:\Langgraph\Agentic_by_langgraph\Sequential_workflow\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [4]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[]